In [1]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(database="202011060_db", user = "postgres", password = "admin", host = "localhost", port = "5432",options="-c search_path=project")

In [2]:
A='floor, room, week, day'
B = A.replace(" ","")
A_list = B.split(',')
A_list

['floor', 'room', 'week', 'day']

In [3]:
#Generate F,S and T table 
#T=original table without aggregate attribute (attributes present in A)(reference table)
#S=table with not null values or null values (all attributes)
#F=whole table
cur = conn.cursor()
table_name = 'data1'

query_F = 'select * from '+table_name
cur.execute(query_F)
ans_F=cur.fetchall()
F = pd.DataFrame(ans_F, columns=['floor','room','week','day','kwh'])

query_T = 'select '+A+ ' from '+table_name
cur.execute(query_T)
ans_T=cur.fetchall()
T = pd.DataFrame(ans_T, columns=['floor','room','week','day'])


query_S = 'select '+A+' from '+table_name+' where kwh is null'
cur.execute(query_S)
ans_S=cur.fetchall()
S = pd.DataFrame(ans_S, columns=['floor','room','week','day'])

cur.close()

In [4]:
#powerSet(list,int)
from itertools import combinations 
def powerSet(list1,level):
    ret_list=[]
    l1=combinations(list1, level)
    for i in list(l1):
        ret_list.append(list(i))
    return ret_list
#powerSet(A_list,2)

In [5]:
#patterns(list,dataframe)
def patterns(list2,df1):
    temp = df1[A_list].copy()
    for i in A_list:
        if i not in list2:
            temp[i] = '*'
    list2 = temp.values.tolist()
    list3 = []
    for i in list2:
        if i not in list3:
            list3.append(i)
    return list3
#patterns(['floor', 'week'],S)

In [6]:
#checkComp(list,dataframe,dataframe)
def checkComp(list1,table1,table2):
    index_tobe_checked = {}
    lenS = 0
    lenT = 0
    for ele in list1:
        if ele != '*':
            index_tobe_checked[A_list[list1.index(ele)]] = ele
    tempS = table1.copy()
    tempT = table2.copy()
    for i in index_tobe_checked:
        tempS = tempS[tempS[i]==index_tobe_checked[i]]
        tempT = tempT[tempT[i]==index_tobe_checked[i]]
        lenS = len(tempS.index)
        lenT = len(tempT.index)
    if lenS==lenT:
        return 1
    else:
        return 0
#checkComp(['*', '*', 'w1', '*'],S,T)    

In [7]:
def checkComp2(list1,table1,table2):
    minimal_list={}
    for i in list1:
        index_tobe_checked = {}
        lenS = 0
        lenT = 0
        for ele in i:
            if ele != '*':
                index_tobe_checked[A_list[i.index(ele)]] = ele
        tempS = table1.copy()
        tempT = table2.copy()
        for k in index_tobe_checked:
            tempS = tempS[tempS[k]==index_tobe_checked[k]]
            tempT = tempT[tempT[k]==index_tobe_checked[k]]
        lenS = len(tempS.index)
        lenT = len(tempT.index)
        if lenS==lenT:
            if len(minimal_list) == 0:
                minimal_list[tuple(i)] = tempS
            else:
                minimal_list_copy =  minimal_list.copy()
                for m in minimal_list_copy:
                    if tempS.isin(minimal_list[m]).all().all():
                        break 
                    elif minimal_list[m].isin(tempS).all().all():
                        minimal_list[tuple(i)] = tempS
                        minimal_list.pop(m)
                    elif tempS.isin(pd.concat(list(minimal_list.values())).drop_duplicates().reset_index(drop=True)).all().all():
                        break
                    else:
                        minimal_list[tuple(i)] = tempS
    minimal_list_pattern = []
    for i in minimal_list:
        minimal_list_pattern.append(list(i))
    return minimal_list_pattern
#checkComp2([['f1', '*', '*', '*'], ['f2', '*', '*', '*'], ['f3', '*', '*', '*'], ['*', 'r1', '*', '*'], ['*', 'r2', '*', '*'], ['*', '*', 'w1', '*'], ['*', '*', 'w2', '*'], ['*', '*', '*', 'Mon '], ['*', '*', '*', 'Tue']],S,T)    

In [8]:
F

,floor,room,week,day,kwh
0,f1,r1,w1,Mon,10.0
1,f1,r1,w1,Tue,12.0
2,f1,r1,w2,Mon,10.0
3,f1,r1,w2,Tue,NaN
4,f1,r2,w1,Mon,8.0
5,f1,r2,w1,Tue,10.0
6,f1,r2,w2,Mon,NaN
7,f1,r2,w2,Tue,NaN
8,f2,r1,w1,Mon,12.0
9,f2,r1,w1,Tue,7.0


In [9]:
S

,floor,room,week,day
0,f1,r1,w2,Tue
1,f1,r2,w2,Mon
2,f1,r2,w2,Tue


In [10]:
T

,floor,room,week,day
0,f1,r1,w1,Mon
1,f1,r1,w1,Tue
2,f1,r1,w2,Mon
3,f1,r1,w2,Tue
4,f1,r2,w1,Mon
5,f1,r2,w1,Tue
6,f1,r2,w2,Mon
7,f1,r2,w2,Tue
8,f2,r1,w1,Mon
9,f2,r1,w1,Tue
